In [1]:
# enable autorreload of modules
%load_ext autoreload
import os
# os.setpriority(
import pickle
import numpy as np
import torch
import matplotlib.pyplot as plt
from new_models import *
from SDE_helper import *
import warnings
# warnings.filterwarnings('always', module='.*')
import logging
# logging.getLogger().setLevel(logging.ERROR)

# 1. Initial Attempt

In [2]:
# noise_level = 1.0
# dg = pickle.load(open("data/datafile_dose_exp_test.pkl", "rb"))
# seed = 666
# torch.manual_seed(seed)
# noise = torch.randn_like(dg.measurements) * (noise_level - 0.2)
# with torch.no_grad():
#     dg.measurements = dg.measurements + noise
#     dg.split_sample()

# with open("data/datafile_dose_noise_{}.pkl".format(noise_level), "wb") as f:
#     pickle.dump(dg, f)

In [5]:
from new_models import SDENet, NewEncoderLSTM, NewVariationalInference
with open('data/datafile_dim8.pkl', "rb") as f:
# with open('data/datafile_dose_noise_1.0.pkl', "rb") as f:
    data_gen = pickle.load(f)
    
data_config = sim_config.dim8_config
# data_config = sim_config.DataConfig(n_sample=1000)

obs_dim = data_config.obs_dim
latent_dim = data_config.latent_dim
action_dim = data_config.action_dim
t_max = data_config.t_max
step_size = data_config.step_size
encoder_latent_ratio = 2.0
encoder_output_dim = 6
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'mps'
normalize = True
prior = ExponentialPrior.log_density
ablate = True
condition_w = True
condition_w_only = False



encoder = NewEncoderLSTM(
            obs_dim + action_dim,
            int(obs_dim * encoder_latent_ratio),
            encoder_output_dim,
            device=device,
            normalize=normalize,
        )


new_decoder = SDENet(
    input_size=(encoder_output_dim,),
    device=device,
    condition_w=condition_w,
    obs_dim=obs_dim,
    y_net_hidden_width=4,
    sigma=0.01,
    dt=0.001,
    weight_network_sizes=(32,), #128
    control_network_width=4, #8
    ablate=ablate,
    condition_w_only=condition_w_only,
)


new_vi = NewVariationalInference(encoder, 
                            new_decoder, 
                            prior_log_pdf=prior, 
                            elbo=True,
                            kl_coeff1=0.2,
                            kl_coeff2=0.2,
                            use_full_mc_kl=False,
                            use_full_mc_lik=False,
                            lik_mc_size=10,
                            mc_size=100,
                            )

if not condition_w:
    new_params = (
                list(new_vi.encoder.parameters())
                + list(new_vi.decoder.y_net.parameters())
                + list(new_vi.decoder.w_net.parameters())
                + list(new_vi.decoder.projection.parameters())
            )
else:
    new_params = (
                list(new_vi.encoder.parameters()) +
                list(new_vi.decoder.y_net.parameters())
                + list(new_vi.decoder.w_net.parameters())
                + list(new_vi.decoder.projection.parameters())
                + list(new_vi.decoder.control_net.parameters())
            
    )

Running ablation study: True
Conditioning on w only: False
Number of parameters of y net: 22
Conditioning on expert ODEs: True
Conditioning expert ODEs only False


In [6]:
from training_utils import *
with open('data/datafile_dim8.pkl', "rb") as f:
    data_gen = pickle.load(f)
dg = data_gen
dg.set_device(device)
dg.set_train_size(1000)
variational_training_loop(
    1000,
    data_gen,
    new_vi,
    64,
    torch.optim.Adam(new_params, lr=1e-6),
    10,
    best_on_disk=1e9,
    early_stop=100,
    path='new-models/',
    print_future_mse=True,
)

2023-11-24 22:17:53,074 Training VI_LSTMEncoder_HybridDecoder.pkl


train_size -100
n_sample 1000
iter 1
iter 2


ValueError: Expected value argument (Tensor of shape (64, 6)) to be within the support (GreaterThanEq(lower_bound=0.0)) of the distribution Exponential(rate: tensor([100.])), but found invalid values:
tensor([[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],

# 2. Second Attempt

In [ ]:
# from new_models import SDENet, NewEncoderLSTM, NewVariationalInference, SDENet2
# with open('data/datafile_dim8.pkl', "rb") as f:
# # with open('data/datafile_dose_noise_1.0.pkl', "rb") as f:
#     data_gen = pickle.load(f)
    
# data_config = sim_config.dim8_config
# # data_config = sim_config.DataConfig(n_sample=1000)

# obs_dim = data_config.obs_dim
# latent_dim = data_config.latent_dim
# action_dim = data_config.action_dim
# t_max = data_config.t_max
# step_size = data_config.step_size
# encoder_latent_ratio = 2.0
# encoder_output_dim = 6
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# # device = 'mps'
# normalize = True
# prior = ExponentialPrior.log_density
# ablate = True
# condition_w = False
# condition_w_only = False



# encoder = NewEncoderLSTM(
#             obs_dim + action_dim,
#             int(obs_dim * encoder_latent_ratio),
#             encoder_output_dim,
#             device=device,
#             normalize=normalize,
#         )


# new_decoder = SDENet2(
#     input_size=(encoder_output_dim,),
#     device=device,
#     obs_dim=obs_dim,
#     y_net_hidden_width=None,
#     sigma=0.1,
#     weight_network_sizes=(1,16, 1), #128
# )


# new_vi = NewVariationalInference(encoder, 
#                             new_decoder, 
#                             prior_log_pdf=prior, 
#                             elbo=True,
#                             kl_coeff1=0.2,
#                             kl_coeff2=0.2,
#                             use_full_mc_kl=False,
#                             use_full_mc_lik=False,
#                             lik_mc_size=10,
#                             mc_size=100,
#                             )

# new_params = (
#             list(new_vi.encoder.parameters())
#             + list(new_vi.decoder.y_net.parameters())
#             + list(new_vi.decoder.w_net.parameters())
#             + list(new_vi.decoder.projection.parameters())
#         )

In [ ]:
# from training_utils import *
# with open('data/datafile_dim8.pkl', "rb") as f:
#     data_gen = pickle.load(f)
# dg = data_gen
# dg.set_device(device)
# dg.set_train_size(1000)
# variational_training_loop(
#     1000,
#     data_gen,
#     new_vi,
#     64,
#     torch.optim.Adam(new_params, lr=1e-2),
#     10,
#     best_on_disk=1e9,
#     early_stop=100,
#     path='new-models/',
#     print_future_mse=True,
# )

In [ ]:
# from training_utils import evaluate
# evaluate(new_vi, 
#          data_gen, 
#          64, 
#          5,
#          mc_itr=50, 
#          real=False)

# # dim fw=(1,64,1), without weights constraint
# # (0.25526687502861023,
# #  0.0013087330489873566,
# #  0.06703939384492985,
# #  0.9209802746772766,
# #  0.006640289111317933,
# #  0.668211308672912)


# 3. Another attempt

In [ ]:
from new_models import SDENet3, NewEncoderLSTM, NewVariationalInference
with open('data/datafile_dim8.pkl', "rb") as f:
# with open('data/datafile_dose_noise_1.0.pkl', "rb") as f:
    data_gen = pickle.load(f)
    
data_config = sim_config.dim8_config
# data_config = sim_config.DataConfig(n_sample=1000)

obs_dim = data_config.obs_dim
latent_dim = data_config.latent_dim
action_dim = data_config.action_dim
t_max = data_config.t_max
step_size = data_config.step_size
encoder_latent_ratio = 2.0
encoder_output_dim = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'mps'
normalize = True
prior = ExponentialPrior.log_density
ablate = False
condition_w = False
condition_w_only = False



encoder = NewEncoderLSTM(
            obs_dim + action_dim,
            int(obs_dim * encoder_latent_ratio),
            encoder_output_dim,
            device=device,
            normalize=normalize,
        )


new_decoder = SDENet3(
    input_size=(encoder_output_dim,),
    device=device,
    obs_dim=obs_dim,
    action_dim=action_dim,
    latent_dim=encoder_output_dim,
    y_net_hidden_width=None,
    batch_size=64,
    sigma=0.01,
    weight_network_sizes=(1,), #128
)


new_vi = NewVariationalInference(encoder, 
                            new_decoder, 
                            prior_log_pdf=prior, 
                            elbo=True,
                            kl_coeff1=0.2,
                            kl_coeff2=0.2,
                            use_full_mc_kl=False,
                            use_full_mc_lik=False,
                            lik_mc_size=10,
                            mc_size=100,
                            )




new_params = (
            list(new_vi.encoder.parameters())
            + list(new_vi.decoder.y_net.parameters())
            + list(new_vi.decoder.controlled_w.parameters())
            + list(new_vi.decoder.projection.parameters())
)


                

In [ ]:
from training_utils import *
with open('data/datafile_dim8.pkl', "rb") as f:
    data_gen = pickle.load(f)
dg = data_gen
dg.set_device(device)
dg.set_train_size(1000)
variational_training_loop(
    1000,
    dg,
    new_vi,
    64,
    torch.optim.Adam(new_params, lr=1e-3),
    10,
    best_on_disk=1e9,
    early_stop=100,
    path='new-models/',
    print_future_mse=True,
)

In [ ]:
# from training_utils import evaluate
# evaluate(new_vi, 
#          data_gen, 
#          64, 
#          5,
#          mc_itr=50, 
#          real=False)

# 4. Another attempt

## Train expert model for latent dynamics

In [ ]:
from model import *
from training_utils import *

with open('data/datafile_dim8.pkl', "rb") as f:
# with open('data/datafile_dose_noise_1.0.pkl', "rb") as f:
    data_gen = pickle.load(f)
    
data_config = sim_config.dim8_config
# data_config = sim_config.DataConfig(n_sample=1000)

obs_dim = data_config.obs_dim
latent_dim = data_config.latent_dim
action_dim = data_config.action_dim
t_max = data_config.t_max
step_size = data_config.step_size
encoder_latent_ratio = 2.0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
normalize = True
prior = ExponentialPrior.log_density
encoder_output_dim = 4

old_encoder = EncoderLSTM(
            obs_dim + action_dim,
            int(obs_dim * encoder_latent_ratio),
            4,
            device=device,
            normalize=normalize,
        )

old_decoder = RocheExpertDecoder(
            obs_dim,
            4,
            action_dim,
            14,
            1,
            roche=True,
            method="dopri5",
            device=device,
            ablate=False,
        )
old_vi = VariationalInference(old_encoder,
                                old_decoder,
                                prior_log_pdf=prior,
                                elbo=True)

params = (
            list(old_vi.encoder.parameters())
            + list(old_vi.decoder.output_function.parameters())
            + list(old_vi.decoder.ode.ml_net.parameters())
        )

variational_training_loop(
    1,
    data_gen,
    old_vi,
    50,
    torch.optim.Adam(params, lr=1e-2),
    5,
    early_stop=10,
    print_future_mse=True,
)

In [ ]:
# evaluate(old_vi,
#             data_gen,
#             50,
#             5,
#             mc_itr=50,
#             real=False)
# (0.026531780138611794,
#  0.0005103847261433787,
#  0.008222115392954869,
#  0.7614899277687073,
#  0.005279327186749542,
#  0.5488908503766188)

## Train latent dynamics model as neural controlled ODE

In [ ]:
# from new_models import NewEncoderLSTM
# from new_models_hybrid import SDENet4, HybridVariationalInference

# with open('data/datafile_dim8.pkl', "rb") as f:
# # with open('data/datafile_dose_noise_1.0.pkl', "rb") as f:
#     data_gen = pickle.load(f)
    
# data_config = sim_config.dim8_config
# # data_config = sim_config.DataConfig(n_sample=1000)

# obs_dim = data_config.obs_dim
# latent_dim = data_config.latent_dim
# action_dim = data_config.action_dim
# t_max = data_config.t_max
# step_size = data_config.step_size
# encoder_latent_ratio = 2.0
# encoder_output_dim = 6
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# # device = 'mps'
# normalize = True
# prior = ExponentialPrior.log_density
# ablate = False


# from model import EncoderLSTM

# old_encoder = EncoderLSTM(
#             obs_dim + action_dim,
#             int(obs_dim * encoder_latent_ratio),
#             4,
#             device=device,
#             normalize=normalize,
#         )
# model_path = 'model/VI_LSTMEncoder_ExpertDecoder.pkl'
# old_vi = torch.load(model_path)
# old_encoder.load_state_dict(torch.load(model_path)['encoder_state_dict'])



# encoder = NewEncoderLSTM(
#             obs_dim + action_dim,
#             int(obs_dim * encoder_latent_ratio),
#             encoder_output_dim,
#             device=device,
#             normalize=normalize,
#         )


# new_decoder = SDENet4(
#     input_size=(encoder_output_dim,),
#     device=device,
#     obs_dim=obs_dim,
#     action_dim=action_dim,
#     latent_dim=encoder_output_dim,
#     y_net_hidden_size=(1,),
#     augmented_dim=4,
#     batch_size=64,
#     sigma=0.01,
#     weight_network_sizes=(1,8,1), #128
# )


# new_vi = HybridVariationalInference(encoder, 
#                             new_decoder, 
#                             prior_log_pdf=prior, 
#                             elbo=True,
#                             kl_coeff1=0.2,
#                             kl_coeff2=0.2,
#                             use_full_mc_kl=False,
#                             use_full_mc_lik=False,
#                             lik_mc_size=10,
#                             mc_size=10,
#                             expert_encoder=old_encoder,
#                             )


# new_params = (
#             list(new_vi.encoder.parameters())
#             + list(new_vi.decoder.y_net.parameters())
#             + list(new_vi.decoder.w_net.parameters())
#             + list(new_vi.decoder.projection.parameters())
# )

In [ ]:
# from training_utils import *
# with open('data/datafile_dim8.pkl', "rb") as f:
#     data_gen = pickle.load(f)
# dg = data_gen
# dg.set_device(device)
# dg.set_train_size(1000)
# variational_training_loop(
#     5000,
#     data_gen,
#     new_vi,
#     64,
#     torch.optim.Adam(new_params, lr=1e-3),
#     10,
#     best_on_disk=1e9,
#     early_stop=100,
#     path='new-models/',
#     print_future_mse=True,
#     hybrid_cde=True,
# )

In [ ]:
# evaluate(new_vi,
#             data_gen,
#             64,
#             5,
#             mc_itr=50,
#             real=False)

# 5. Further attempt (include expert dimension but don't use it)


## Exploring the latent space

In [ ]:
data = data_gen.get_mini_batch("train", 100)
x = data["measurements"]
a = data["actions"]
mask = data["masks"]
z0 = data["latents"]
z0 = z0.numpy()
print(z0.shape)

## Exploring the numerical error

In [ ]:
from scipy.integrate import solve_ivp
import numpy as np
import matplotlib.pyplot as plt

# Example differential equations
def f(t, x):
    return np.cos(t) - x

def g(t, y):
    # return np.cos(t) - y
    return np.cos(t) - y

# Integrating equation 1 over [0, 1] with time scaled by 14
t1_span = (0, 1)
t1_points = np.linspace(*t1_span, 100)
solution1 = solve_ivp(lambda t, x: f(14*t, x), t1_span, [0], t_eval=t1_points)

# Integrating equation 2 over [0, 14]
t2_span = (0, 14)
t2_points = np.linspace(*t2_span, 100)
solution2 = solve_ivp(g, t2_span, [0], t_eval=t2_points)

# Comparing solutions
# Scaling t1_points by 14 to match the time points with t2_points for comparison
scaled_t2_points = t2_points / 14

# Plotting the solutions
plt.figure(figsize=(12, 6))
plt.plot(t1_points, solution1.y[0], label='Equation 1 (f(t, x))')
plt.plot(scaled_t2_points, solution2.y[0]/14, label='Equation 2 (g(t/14, y))', linestyle='dashed')
plt.title('Comparison of Differential Equations')
plt.xlabel('Time')
plt.ylabel('Solution')
plt.legend()
plt.grid(True)
plt.show()